<a href="https://colab.research.google.com/github/kunalom/stock_analysis_RAG/blob/main/stock_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

y
Python 3.11.13


# Installing dependencies

In [ ]:
pip install langchain langchain_huggingface langchain_community faiss-cpu pypdf beautifulsoup4 requests

y
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7

In [ ]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_hugging face_api"

## just checking model is working or not

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

result = model.invoke("What is the capital of India?")

print(result.content)

The capital of India is New Delhi.


In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace
from langchain.schema import Document

## Function to scrape Screener.in

In [ ]:
def scrape_screener_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    text = soup.get_text(separator="\n", strip=True)
    return text

## Function to create augmented prompt

In [ ]:
def create_augmented_prompt(stock_data, question):
    return f"""You are a stock market expert trained on high-quality investment material.

Given the following stock information scraped from Screener.in and the investment strategies you've learned from the PDFs:

---
STOCK DATA:
{stock_data}

QUESTION:
{question}
"""

## Function to load and index PDFs

In [ ]:
def load_and_index_pdfs(pdf_paths):
    documents = []
    for path in pdf_paths:
        loader = PyPDFLoader(path)
        docs = loader.load()
        documents.extend(docs)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore

# Function to create retriever

In [ ]:
def create_retriever(vectorstore):
    return vectorstore.as_retriever()


## Function to run RAG analysis

In [ ]:
def run_rag_analysis(vectorstore, stock_url, model):
    stock_text = scrape_screener_data(stock_url)
    retriever = create_retriever(vectorstore)

    prompt = create_augmented_prompt(
        stock_data=stock_text,
        question="Should I invest in this stock? What are the pros and cons based on your knowledge?"
    )

    qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever, chain_type="stuff")
    answer = qa_chain.invoke(prompt)
    return answer

In [ ]:
stock_url = "https://www.screener.in/company/SBILIFE/"

## Load PDFs and run RAG

In [ ]:
pdfs = ["/content/Module 1_Introduction to Stock Markets.pdf",
         "/content/Module 3_Fundamental Analysis.pdf"
            ]
vectorstore = load_and_index_pdfs(pdfs)
result = run_rag_analysis(vectorstore, stock_url, model)

#Result

In [ ]:
print(result['result'])

<ipython-input-6-70375c4fcb9f>:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Based on the provided financial data and analysis tools, I can offer some insights on the pros and cons of investing in SBI Life Insurance Company Ltd. Please note that this is not personalized investment advice, and it's essential to do your own research and consider your financial goals and risk tolerance before making any investment decisions.

**Pros:**

1. **Debt-free company**: SBI Life Insurance Company Ltd has minimal debt, which reduces its financial risk and allows it to allocate more resources to investing and growing its business.
2. **Strong brand and presence**: As a joint venture between State Bank of India and BNP Paribas Cardif S.A., SBI Life Insurance has a well-established brand and wide reach, which can help attract customers and increase sales.
3. **Diversified product portfolio**: The company offers a range of life insurance products, including individual and group plans, which can help spread risk and provide opportunities for growth.
4. **Growing premium income*